In [1]:
import glob
import os
import numpy as np
from PIL import Image
import skimage
import skimage.io
import scipy.misc
import imageio
import warnings; warnings.simplefilter('ignore')

list_classes = [
 'Sony-NEX-7',
 'Motorola-X',
 'HTC-1-M7',
 'Samsung-Galaxy-Note3',
 'Motorola-Droid-Maxx',
 'iPhone-4s',
 'iPhone-6',
 'LG-Nexus-5x',
 'Samsung-Galaxy-S4',
 'Motorola-Nexus-6']

list_dict = {}
for i in range(10):
    key = list_classes[i]
    v = [0,0,0,0,0,0,0,0,0,0]
    v[i] = 1
    list_dict[key] = v
print(list_dict)

train_dir = '../input/train'
test_dir = '../input/test'
test_files = sorted(glob.glob(test_dir+'/*'))
train_files = sorted(glob.glob(train_dir+'/*/*'))
train_data_cnt = len(train_files)
BATCH_SIZE = 13
CROP_LEN = 224

def random_crop(im_array):
    # crop
    x_range = im_array.shape[0] - CROP_LEN
    y_range = im_array.shape[1] - CROP_LEN
    # print(x_range,y_range)
    a = np.random.randint(x_range)
    b = a + CROP_LEN
    c = np.random.randint(y_range)
    d = c + CROP_LEN
    new_im_array = im_array[a:b,c:d,:]
    return new_im_array

def center_crop(im_array):
    center_x = im_array.shape[0] // 2
    center_y = im_array.shape[1] // 2
    half_crop = CROP_LEN // 2
    a = center_x - half_crop
    b = a + CROP_LEN
    c = center_y - half_crop
    d = c + CROP_LEN
    new_im_array = im_array[a:b,c:d,:]
    return new_im_array
    

def random_manip(img,rnd):
    if rnd == 0:
        return img
    
    elif rnd == 1:
        # gamma 0.8
        new_img = skimage.exposure.adjust_gamma(img, gamma=0.8)
        return new_img
    
    elif rnd == 2:
        # gamma 1.2
        new_img = skimage.exposure.adjust_gamma(img, gamma=1.2)
        return new_img
    
    elif rnd == 3:
        # jpeg 70
        imageio.imwrite('/tmp/quality-70.jpg', img, quality=70)
        try:
            new_img = np.array(Image.open(('/tmp/quality-70.jpg')), dtype="uint8")
            os.remove('/tmp/quality-70.jpg')
            return new_img
        except:
            return img
    
    elif rnd == 4:
        # jpeg 90
        imageio.imwrite('/tmp/quality-90.jpg', img, quality=90)
        try:
            new_img = np.array(Image.open(('/tmp/quality-90.jpg')), dtype="uint8")
            os.remove('/tmp/quality-90.jpg')
            return new_img
        except:
            return img
    
    elif rnd == 5:
        # 2x of original image
        new_img = scipy.misc.imresize(img, 2.0, interp='bicubic')
        return center_crop(new_img)
    
    elif rnd == 6:
        new_img = scipy.misc.imresize(img, 1.5, interp='bicubic')
        return center_crop(new_img)
    
    elif rnd == 7:
        new_img = scipy.misc.imresize(img, 0.5, interp='bicubic')
        return new_img
    
    elif rnd == 8:
        new_img = scipy.misc.imresize(img, 0.8, interp='bicubic')
        return new_img
    
    else:
        return img

{'Motorola-X': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'Samsung-Galaxy-S4': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 'iPhone-6': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 'Samsung-Galaxy-Note3': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 'Motorola-Nexus-6': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 'LG-Nexus-5x': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 'Motorola-Droid-Maxx': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 'HTC-1-M7': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 'Sony-NEX-7': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'iPhone-4s': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]}


In [2]:
def get_img(img_path, train_flag = True, fake_rnd = 0, rnd_range = 16):
    # read img
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    
    # train or valid
    if train_flag is True:
        # manip rnd
        manip_rnd = np.random.randint(rnd_range) # only half manip, 1 to 8 use manip, if rnd_range = 64, 1/8 manip
        #manip_rnd = 5
        #print(manip_rnd,im_array.shape)
        if manip_rnd < 7 or manip_rnd > 8 : 
            # no zoom in， 随机切出 224
            im_array = random_crop(im_array)
            # manip， 随机变化，如果放大则去中间部分
            final_img = random_manip(im_array, manip_rnd)
          
        else:
            # resize zoom out， 缩小
            im_array = random_manip(im_array, manip_rnd)
            # random crop on larger image， 随机切出 224
            final_img = random_crop(im_array)
        if final_img.shape[0]!=CROP_LEN or final_img.shape[1]!=CROP_LEN:
            print('train',manip_rnd,final_img.shape,img_path,im_array.shape)
    else:
        # center crop for valid data
        # manip rnd
        manip_rnd = fake_rnd
        if manip_rnd < 7 or manip_rnd > 8 :
            # no zoom in， 取中间部分
            im_array = center_crop(im_array)
            # manip, 随机变化，如果放大则取中间部分
            final_img = random_manip(im_array, manip_rnd)
        else:
            # resize zoom out, 先缩小图片
            im_array = random_manip(im_array, manip_rnd)
            # random crop on larger image, 取中间部分
            final_img = center_crop(im_array)
        if final_img.shape[0]!=CROP_LEN or final_img.shape[1]!=CROP_LEN:
            print('valid',manip_rnd,final_img.shape,img_path,im_array.shape)
    
    final_img = final_img/127.5
    final_img = final_img - 1.0
    return final_img

# 读取测试数据中间 224
def get_test_img(img_path):
    # read img
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    final_img = center_crop(im_array)
    final_img = final_img/127.5
    final_img = final_img - 1.0
    return final_img

def get_y(img_path):
    n = img_path.split('/')[-2]
    return list_dict[n]

def data_gen(file_list, batch_size=BATCH_SIZE, train_flag = True, manip_rnd_range = 16):
    curr_idx = 0
    data_cnt = len(file_list)
    fake_rnd = 0
    while True:
        if curr_idx + batch_size > data_cnt:
            start_idx = data_cnt-batch_size
            end_idx = data_cnt
            curr_idx = 0
            fake_rnd = 0 # reset fake_rnd, 13 不是 16 的整数倍, 从第二次 valid 开始变得 stable
        else:
            start_idx = curr_idx
            end_idx = curr_idx + batch_size
            curr_idx += batch_size
        curr_fl = file_list[start_idx:end_idx]
        if train_flag is True:
            curr_x = [get_img(p,train_flag,fake_rnd=0,rnd_range=manip_rnd_range) for p in curr_fl]
        else:
            # make validation data stable
            curr_x = []
            for p in curr_fl:
                tmp_img = get_img(p,train_flag,fake_rnd)
                curr_x.append(tmp_img)
                fake_rnd += 1
                fake_rnd = fake_rnd % manip_rnd_range
        curr_x = np.array(curr_x,dtype='float32')
        curr_y = np.array([get_y(p) for p in curr_fl])
        yield curr_x,curr_y

from sklearn.utils import shuffle
train_files = shuffle(train_files,random_state=42)

# 1/8 manip
train_less_manip_gen = data_gen(train_files, BATCH_SIZE, True, 64)
valid_less_manip_gen = data_gen(train_files, BATCH_SIZE, False, 64)
# 1/2 manip
train_gen = data_gen(train_files, BATCH_SIZE, True, 16)
valid_gen = data_gen(train_files, BATCH_SIZE, False, 16)
import math
train_step = math.ceil(train_data_cnt/BATCH_SIZE)
valid_step = train_step
    

In [3]:
# def model
from keras.models import Sequential,load_model,Model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.


In [4]:
model_p = 'best_resnet_manip.h5'

if os.path.exists(model_p):
    model = load_model(model_p)
    K.set_value(model.optimizer.lr, 0.0001)
    print('load',model_p)
else:
    from keras.applications.resnet50 import ResNet50
    base_model = ResNet50(input_shape=(CROP_LEN,CROP_LEN,3),include_top=False,weights=None,pooling='max')
    x = base_model.output
    x = Dense(256,activation='relu')(x)
    pred = Dense(10,activation='softmax')(x)
    model = Model(inputs=base_model.input,outputs=pred)
    print('get model')
    print(model.summary())
    model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0001), metrics=['accuracy'])

load best_resnet_manip.h5


In [5]:
    
from keras import backend as K
#K.set_value(model.optimizer.lr, 0.0005)   

model_chk = ModelCheckpoint(filepath=model_p, monitor='val_acc', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, verbose=1,
                              patience=3, min_lr=0.00000001)
# less manip
model.fit_generator(train_less_manip_gen,
          steps_per_epoch = train_step,
          epochs=150,
          callbacks=[reduce_lr]
         )
model.save('less_manip_resnet.h5')
# half manip
K.set_value(model.optimizer.lr, 0.0001)
model.fit_generator(train_gen,
          steps_per_epoch = train_step,
          epochs=150,
          validation_data = valid_gen,
          validation_steps = valid_step,
          callbacks=[model_chk,reduce_lr]
         )

Epoch 1/150
212/212 [==============================] - 344s 2s/step - loss: 1.5185 - acc: 0.4771
Epoch 2/150
212/212 [==============================] - 333s 2s/step - loss: 1.5780 - acc: 0.4554
Epoch 3/150
212/212 [==============================] - 336s 2s/step - loss: 1.5250 - acc: 0.4663
Epoch 4/150
212/212 [==============================] - 333s 2s/step - loss: 1.5015 - acc: 0.4913
Epoch 5/150
212/212 [==============================] - 334s 2s/step - loss: 1.4715 - acc: 0.4895
Epoch 6/150
212/212 [==============================] - 332s 2s/step - loss: 1.4788 - acc: 0.4917
Epoch 7/150
212/212 [==============================] - 332s 2s/step - loss: 1.4279 - acc: 0.5000
Epoch 8/150
212/212 [==============================] - 332s 2s/step - loss: 1.4833 - acc: 0.4873
Epoch 9/150
212/212 [==============================] - 326s 2s/step - loss: 1.4284 - acc: 0.5127
Epoch 10/150
212/212 [==============================] - 333s 2s/step - loss: 1.4330 - acc: 0.5065
Epoch 11/150
212/212 [=======

212/212 [==============================] - 328s 2s/step - loss: 0.6966 - acc: 0.7783
Epoch 78/150
212/212 [==============================] - 330s 2s/step - loss: 0.6768 - acc: 0.7801
Epoch 79/150
212/212 [==============================] - 328s 2s/step - loss: 0.6614 - acc: 0.7816
Epoch 80/150
212/212 [==============================] - 327s 2s/step - loss: 0.6597 - acc: 0.7758
Epoch 81/150
212/212 [==============================] - 326s 2s/step - loss: 0.6400 - acc: 0.7877
Epoch 82/150
212/212 [==============================] - 327s 2s/step - loss: 0.6482 - acc: 0.7830
Epoch 83/150
212/212 [==============================] - 332s 2s/step - loss: 0.6451 - acc: 0.7903
Epoch 84/150
212/212 [==============================] - 332s 2s/step - loss: 0.6311 - acc: 0.7925
Epoch 85/150
212/212 [==============================] - 336s 2s/step - loss: 0.6712 - acc: 0.7783
Epoch 86/150
212/212 [==============================] - 327s 2s/step - loss: 0.6856 - acc: 0.7808
Epoch 87/150
212/212 [===========

212/212 [==============================] - 327s 2s/step - loss: 0.6576 - acc: 0.7859
Epoch 148/150
212/212 [==============================] - 328s 2s/step - loss: 0.6264 - acc: 0.7954
Epoch 149/150
212/212 [==============================] - 331s 2s/step - loss: 0.6124 - acc: 0.7935
Epoch 150/150
212/212 [==============================] - 329s 2s/step - loss: 0.6534 - acc: 0.7856
Epoch 1/150
212/212 [==============================] - 828s 4s/step - loss: 1.1969 - acc: 0.5929 - val_loss: 1.8037 - val_acc: 0.5142
Epoch 2/150
212/212 [==============================] - 810s 4s/step - loss: 1.1810 - acc: 0.6081 - val_loss: 1.7485 - val_acc: 0.5025
Epoch 3/150
212/212 [==============================] - 818s 4s/step - loss: 1.1883 - acc: 0.5947 - val_loss: 1.5871 - val_acc: 0.4964
Epoch 4/150
212/212 [==============================] - 816s 4s/step - loss: 1.1415 - acc: 0.6197 - val_loss: 1.8598 - val_acc: 0.4597
Epoch 5/150
212/212 [==============================] - 819s 4s/step - loss: 1.1902

Epoch 30/150
212/212 [==============================] - 810s 4s/step - loss: 0.8572 - acc: 0.7086 - val_loss: 1.1611 - val_acc: 0.6306
Epoch 31/150
212/212 [==============================] - 816s 4s/step - loss: 0.8670 - acc: 0.7054 - val_loss: 0.8528 - val_acc: 0.7177
Epoch 32/150
212/212 [==============================] - 816s 4s/step - loss: 0.8302 - acc: 0.7261 - val_loss: 0.9347 - val_acc: 0.7046
Epoch 33/150
212/212 [==============================] - 814s 4s/step - loss: 0.8577 - acc: 0.7177 - val_loss: 0.7860 - val_acc: 0.7427
Epoch 34/150
212/212 [==============================] - 814s 4s/step - loss: 0.8533 - acc: 0.7094 - val_loss: 0.8798 - val_acc: 0.7130
Epoch 35/150
212/212 [==============================] - 808s 4s/step - loss: 0.8073 - acc: 0.7319 - val_loss: 0.9397 - val_acc: 0.6974
Epoch 36/150
212/212 [==============================] - 810s 4s/step - loss: 0.8311 - acc: 0.7192 - val_loss: 0.9455 - val_acc: 0.6927
Epoch 37/150
212/212 [==============================] -

212/212 [==============================] - 816s 4s/step - loss: 0.7041 - acc: 0.7627 - val_loss: 0.7158 - val_acc: 0.7779
Epoch 62/150
212/212 [==============================] - 818s 4s/step - loss: 0.7161 - acc: 0.7642 - val_loss: 0.6763 - val_acc: 0.7830
Epoch 63/150
212/212 [==============================] - 822s 4s/step - loss: 0.7191 - acc: 0.7522 - val_loss: 0.6409 - val_acc: 0.7964
Epoch 64/150
212/212 [==============================] - 811s 4s/step - loss: 0.7164 - acc: 0.7558 - val_loss: 0.6514 - val_acc: 0.7910
Epoch 65/150
211/212 [============================>.] - ETA: 1s - loss: 0.7073 - acc: 0.7656Epoch 00065: val_acc did not improve

Epoch 00065: reducing learning rate to 3.12499992105586e-06.
212/212 [==============================] - 814s 4s/step - loss: 0.7077 - acc: 0.7656 - val_loss: 0.6663 - val_acc: 0.7899
Epoch 66/150
212/212 [==============================] - 810s 4s/step - loss: 0.6738 - acc: 0.7758 - val_loss: 0.6428 - val_acc: 0.8044
Epoch 67/150
212/212 [===

Epoch 91/150
212/212 [==============================] - 816s 4s/step - loss: 0.6702 - acc: 0.7801 - val_loss: 0.6146 - val_acc: 0.8095
Epoch 92/150
211/212 [============================>.] - ETA: 1s - loss: 0.7000 - acc: 0.7685Epoch 00092: val_acc did not improve

Epoch 00092: reducing learning rate to 9.765624753299562e-08.
212/212 [==============================] - 814s 4s/step - loss: 0.6995 - acc: 0.7689 - val_loss: 0.6151 - val_acc: 0.8088
Epoch 93/150
212/212 [==============================] - 808s 4s/step - loss: 0.6778 - acc: 0.7700 - val_loss: 0.6172 - val_acc: 0.8095
Epoch 94/150
212/212 [==============================] - 811s 4s/step - loss: 0.6836 - acc: 0.7725 - val_loss: 0.6162 - val_acc: 0.8088
Epoch 95/150
211/212 [============================>.] - ETA: 1s - loss: 0.6780 - acc: 0.7794Epoch 00095: val_acc did not improve

Epoch 00095: reducing learning rate to 4.882812376649781e-08.
212/212 [==============================] - 821s 4s/step - loss: 0.6774 - acc: 0.7798 - va

212/212 [==============================] - 817s 4s/step - loss: 0.7083 - acc: 0.7663 - val_loss: 0.6161 - val_acc: 0.8062
Epoch 123/150
212/212 [==============================] - 817s 4s/step - loss: 0.6612 - acc: 0.7783 - val_loss: 0.6138 - val_acc: 0.8073
Epoch 124/150
212/212 [==============================] - 817s 4s/step - loss: 0.6783 - acc: 0.7721 - val_loss: 0.6148 - val_acc: 0.8102
Epoch 125/150
212/212 [==============================] - 812s 4s/step - loss: 0.6921 - acc: 0.7743 - val_loss: 0.6126 - val_acc: 0.8088
Epoch 126/150
212/212 [==============================] - 805s 4s/step - loss: 0.6774 - acc: 0.7674 - val_loss: 0.6152 - val_acc: 0.8066
Epoch 127/150
212/212 [==============================] - 817s 4s/step - loss: 0.7186 - acc: 0.7620 - val_loss: 0.6174 - val_acc: 0.8088
Epoch 128/150
212/212 [==============================] - 809s 4s/step - loss: 0.6929 - acc: 0.7689 - val_loss: 0.6165 - val_acc: 0.8055
Epoch 129/150
212/212 [==============================] - 811s 

In [6]:
best_model = load_model(model_p)
test_y = []
for img_p in test_files:
    tmp_x = get_test_img(img_p)
    tmp_y = best_model.predict(np.array([tmp_x]))[0]
    test_y.append(tmp_y)

test_y = np.array(test_y)
print(test_y[:5])

[[  2.27566739e-03   6.45841509e-02   2.97193159e-03   1.05084278e-01
    5.25144441e-03   9.82298225e-05   1.26907513e-01   1.79201141e-01
    7.80865699e-02   4.35539067e-01]
 [  4.25943546e-02   1.17025222e-03   8.55111610e-03   1.15131226e-03
    9.80214961e-03   7.80220687e-01   6.19331971e-02   5.46191819e-02
    1.42819155e-03   3.85296680e-02]
 [  2.61258124e-03   5.43727147e-05   1.01568773e-02   7.53344139e-05
    1.42839548e-04   3.19137424e-03   9.76629853e-01   5.54667879e-03
    5.18945162e-04   1.07117300e-03]
 [  2.36630640e-05   9.30087059e-04   1.88503094e-04   1.22046387e-02
    2.80866770e-05   2.56331696e-04   2.58070906e-03   4.07045186e-02
    9.38252628e-01   4.83088940e-03]
 [  4.04075280e-08   9.92269397e-01   1.21473153e-04   2.83636176e-03
    1.39051481e-04   3.74372961e-04   4.34334106e-05   1.24712766e-03
    2.85401498e-03   1.14670707e-04]]


In [7]:
import pandas as pd
y_res = np.argmax(test_y,axis=1)
y_res = [list_classes[i] for i in y_res]
df = pd.read_csv('../input/sample_submission.csv')
print(df.head())
f_name = [p.split('/')[-1] for p in test_files]
df['fname'] = f_name
df['camera'] = y_res
print(df.head())
df.to_csv('../results/resnet_manip.csv',index=False)

                   fname    camera
0  img_0002a04_manip.tif  iPhone-6
1  img_001e31c_unalt.tif  iPhone-6
2  img_00275cf_manip.tif  iPhone-6
3  img_0034113_unalt.tif  iPhone-6
4  img_00344b7_unalt.tif  iPhone-6
                   fname             camera
0  img_0002a04_manip.tif   Motorola-Nexus-6
1  img_001e31c_unalt.tif          iPhone-4s
2  img_00275cf_manip.tif           iPhone-6
3  img_0034113_unalt.tif  Samsung-Galaxy-S4
4  img_00344b7_unalt.tif         Motorola-X


In [8]:
# TTA
def get_tta_img(img_path):
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    im_array = im_array/127.5
    im_array = im_array - 1.0
    img_list = []
    img_list.append(center_crop(im_array))
    img_list.append(im_array[0:CROP_LEN,0:CROP_LEN,:])
    img_list.append(im_array[0:CROP_LEN,512-CROP_LEN:512,:])
    img_list.append(im_array[512-CROP_LEN:512,0:CROP_LEN,:])
    img_list.append(im_array[512-CROP_LEN:512,512-CROP_LEN:512,:])
    return np.array(img_list)
tta_img = get_tta_img(test_files[0])
print(tta_img.shape)
tta_res = best_model.predict(tta_img)
print(tta_res.shape)
print(np.sum(tta_res,axis=0))

test_y = []
for img_p in test_files:
    tmp_x = get_tta_img(img_p)
    tmp_y = best_model.predict(tmp_x)
    tmp_y = np.sum(tmp_y,axis=0)
    test_y.append(tmp_y)

test_y = np.array(test_y)
print(test_y[:5])

y_res = np.argmax(test_y,axis=1)
y_res = [list_classes[i] for i in y_res]
df = pd.read_csv('../input/sample_submission.csv')
f_name = [p.split('/')[-1] for p in test_files]
df['fname'] = f_name
df['camera'] = y_res
print(df.head())
df.to_csv('../results/resnet_manip_tta.csv',index=False)

(5, 224, 224, 3)
(5, 10)
[ 0.2845816   0.23909293  0.26623553  0.34260133  0.09864068  0.0268707
  1.54612064  0.58612871  0.63078678  0.97894108]
[[  2.84581602e-01   2.39092931e-01   2.66235530e-01   3.42601329e-01
    9.86406803e-02   2.68707015e-02   1.54612064e+00   5.86128712e-01
    6.30786777e-01   9.78941083e-01]
 [  2.06692398e-01   5.27044497e-02   1.13730408e-01   5.18424669e-03
    1.58593237e-01   3.42289448e+00   2.87252784e-01   2.35980481e-01
    1.02664940e-02   5.06700754e-01]
 [  4.61556017e-02   3.98935564e-03   1.44330502e-01   8.28277087e-04
    4.16976353e-03   5.28063672e-03   4.75948572e+00   3.32672596e-02
    1.05032523e-03   1.44221005e-03]
 [  4.15783259e-04   1.32368997e-01   5.43155475e-03   1.04007564e-01
    2.21762853e-03   3.74690769e-03   3.33249494e-02   1.33390844e-01
    4.48504543e+00   1.00050122e-01]
 [  5.99006162e-05   4.59383774e+00   4.23500687e-02   6.84137195e-02
    2.92387325e-03   1.01464251e-02   1.36077078e-03   3.09350751e-02
    2